In [1]:
import findspark
findspark.init()
import sys
from time import time
from pyspark import SparkContext,SparkConf

In [3]:
master = "local"
if len(sys.argv) == 2:
    master = sys.argv[1]
conf = SparkConf().setMaster(master).setAppName("rdd-key-value")
sc = SparkContext(conf=conf)

In [5]:
file_path = "../data/kddcup.data_10_percent.gz"
raw_data = sc.textFile(file_path)
raw_data.take(1),type(raw_data) # pyspark.rdd.RDD

(['0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.00,0.00,0.00,0.00,1.00,0.00,0.00,9,9,1.00,0.00,0.11,0.00,0.00,0.00,0.00,0.00,normal.'],
 pyspark.rdd.RDD)

#### 利用现有数据创建出一个pair rdd

In [6]:
csv_data = raw_data.map(lambda x:x.split(","))
type(csv_data),csv_data.take(2)

(pyspark.rdd.PipelinedRDD,
 [['0',
   'tcp',
   'http',
   'SF',
   '181',
   '5450',
   '0',
   '0',
   '0',
   '0',
   '0',
   '1',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '8',
   '8',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '1.00',
   '0.00',
   '0.00',
   '9',
   '9',
   '1.00',
   '0.00',
   '0.11',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   'normal.'],
  ['0',
   'tcp',
   'http',
   'SF',
   '239',
   '486',
   '0',
   '0',
   '0',
   '0',
   '0',
   '1',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '8',
   '8',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '1.00',
   '0.00',
   '0.00',
   '19',
   '19',
   '1.00',
   '0.00',
   '0.05',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   'normal.']])

In [7]:
key_value_data = csv_data.map(lambda x:(x[41],x))
type(key_value_data),key_value_data.take(2)

(pyspark.rdd.PipelinedRDD,
 [('normal.',
   ['0',
    'tcp',
    'http',
    'SF',
    '181',
    '5450',
    '0',
    '0',
    '0',
    '0',
    '0',
    '1',
    '0',
    '0',
    '0',
    '0',
    '0',
    '0',
    '0',
    '0',
    '0',
    '0',
    '8',
    '8',
    '0.00',
    '0.00',
    '0.00',
    '0.00',
    '1.00',
    '0.00',
    '0.00',
    '9',
    '9',
    '1.00',
    '0.00',
    '0.11',
    '0.00',
    '0.00',
    '0.00',
    '0.00',
    '0.00',
    'normal.']),
  ('normal.',
   ['0',
    'tcp',
    'http',
    'SF',
    '239',
    '486',
    '0',
    '0',
    '0',
    '0',
    '0',
    '1',
    '0',
    '0',
    '0',
    '0',
    '0',
    '0',
    '0',
    '0',
    '0',
    '0',
    '8',
    '8',
    '0.00',
    '0.00',
    '0.00',
    '0.00',
    '1.00',
    '0.00',
    '0.00',
    '19',
    '19',
    '1.00',
    '0.00',
    '0.05',
    '0.00',
    '0.00',
    '0.00',
    '0.00',
    '0.00',
    'normal.'])])

#### pair rdd算子

#### reduceByKey

In [8]:
key_value_duration = csv_data.map(lambda x:(x[41],float(x[0])))
duration_by_key = key_value_duration.reduceByKey(lambda x,y:x+y)
duration_by_key.collect()

[('normal.', 21075991.0),
 ('buffer_overflow.', 2751.0),
 ('loadmodule.', 326.0),
 ('perl.', 124.0),
 ('neptune.', 0.0),
 ('smurf.', 0.0),
 ('guess_passwd.', 144.0),
 ('pod.', 0.0),
 ('teardrop.', 0.0),
 ('portsweep.', 1991911.0),
 ('ipsweep.', 43.0),
 ('land.', 0.0),
 ('ftp_write.', 259.0),
 ('back.', 284.0),
 ('imap.', 72.0),
 ('satan.', 64.0),
 ('phf.', 18.0),
 ('nmap.', 0.0),
 ('multihop.', 1288.0),
 ('warezmaster.', 301.0),
 ('warezclient.', 627563.0),
 ('spy.', 636.0),
 ('rootkit.', 1008.0)]

#### sortBy(fun,True)

In [10]:
duration_sortBy_key = duration_by_key.sortBy(lambda x:x[0],True) # 按照key升序排序
duration_sortBy_key.collect()

[('warezmaster.', 301.0),
 ('warezclient.', 627563.0),
 ('teardrop.', 0.0),
 ('spy.', 636.0),
 ('smurf.', 0.0),
 ('satan.', 64.0),
 ('rootkit.', 1008.0),
 ('portsweep.', 1991911.0),
 ('pod.', 0.0),
 ('phf.', 18.0),
 ('perl.', 124.0),
 ('normal.', 21075991.0),
 ('nmap.', 0.0),
 ('neptune.', 0.0),
 ('multihop.', 1288.0),
 ('loadmodule.', 326.0),
 ('land.', 0.0),
 ('ipsweep.', 43.0),
 ('imap.', 72.0),
 ('guess_passwd.', 144.0),
 ('ftp_write.', 259.0),
 ('buffer_overflow.', 2751.0),
 ('back.', 284.0)]

#### countByKey()算子是Action操作

In [12]:
counts_by_key = key_value_data.countByKey() # defaultdict格式
type(counts_by_key),counts_by_key

(collections.defaultdict,
 defaultdict(int,
             {'normal.': 97278,
              'buffer_overflow.': 30,
              'loadmodule.': 9,
              'perl.': 3,
              'neptune.': 107201,
              'smurf.': 280790,
              'guess_passwd.': 53,
              'pod.': 264,
              'teardrop.': 979,
              'portsweep.': 1040,
              'ipsweep.': 1247,
              'land.': 21,
              'ftp_write.': 8,
              'back.': 2203,
              'imap.': 12,
              'satan.': 1589,
              'phf.': 4,
              'nmap.': 231,
              'multihop.': 7,
              'warezmaster.': 20,
              'warezclient.': 1020,
              'spy.': 2,
              'rootkit.': 10}))

In [14]:
type(counts_by_key.keys())

dict_keys

In [23]:
for i in counts_by_key.keys():
#     print(i,",",counts_by_key[i])
    print(i + "->" + str(counts_by_key[i]))

normal.->97278
buffer_overflow.->30
loadmodule.->9
perl.->3
neptune.->107201
smurf.->280790
guess_passwd.->53
pod.->264
teardrop.->979
portsweep.->1040
ipsweep.->1247
land.->21
ftp_write.->8
back.->2203
imap.->12
satan.->1589
phf.->4
nmap.->231
multihop.->7
warezmaster.->20
warezclient.->1020
spy.->2
rootkit.->10


#### combineByKey

python中格式化字符串

In [28]:
s = "hello pyspark"
print(f"{s}")

hello pyspark
